## Importing Libraries

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
data_path=r'/Users/Lilian/Downloads/Data Analytics Course/Data Analytics Immersion/Acheivement 4/04-2024 Instacart Basket Analysis/'

In [3]:
orders_products_merge = pd.read_pickle(os.path.join(data_path,'02 Data','Prepared Data','df_ords_prods_merge_new_variables.pkl'))

### 2. In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [4]:
orders_products_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code

#### the mean values for the entire data set differs in values in terms of some being higher or lower from the limited dataset, and that is because wider range of values are being calculated which can have an impact on the resulting values.

### 4. Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

#### Creating a new column for max order which will represent maximum order values for each user, grouped by user_id

In [5]:
orders_products_merge['max_order'] = orders_products_merge.groupby(['user_id'])['order_number'].transform('max')

In [6]:
orders_products_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,busiest_days,busiest_period_of_day,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,11,3.0,5,0,both,Regularly busy,Most orders,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,17,20.0,1,1,both,Regularly busy,Most orders,32
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,21,6.0,20,0,both,Busiest days,Average orders,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,13,NaN,10,0,both,Slowest days,Most orders,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,17,9.0,11,1,both,Slowest days,Most orders,3


#### creating a loyalty flag based on the max_order value in order to label each customer as 'loyal' , 'new' or 'regular' customers

In [7]:
orders_products_merge.loc[orders_products_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
orders_products_merge.loc[(orders_products_merge['max_order'] <= 40) & (orders_products_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
orders_products_merge.loc[orders_products_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [8]:
orders_products_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

### 5. The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [9]:
orders_products_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

#### 6. The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
##### If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
##### If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [10]:
orders_products_merge['average_price_spent'] = orders_products_merge.groupby(['user_id'])['prices'].transform('mean')

### orders_products_merge[['user_id','average_price_spent']].head()

In [11]:
orders_products_merge.loc[orders_products_merge['average_price_spent'] < 10,'Spending_flag'] = 'Low spender'
orders_products_merge.loc[(orders_products_merge['average_price_spent'] >= 10),'Spending_flag'] = 'High spender'

In [12]:
orders_products_merge['Spending_flag'].value_counts(dropna = False)

Spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

#### 7. In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
##### If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
##### If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
##### If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [26]:
orders_products_merge['ordering_median'] = orders_products_merge.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [27]:
orders_products_merge[['user_id','ordering_median']].head()

,user_id,ordering_median
0,138,8.0
1,138,8.0
2,709,8.0
3,764,9.0
4,764,9.0


In [28]:
orders_products_merge.loc[orders_products_merge['ordering_median']> 20 ,'Customer_order_frequency'] = 'Non-frequent customer'

In [29]:

orders_products_merge.loc[(orders_products_merge['ordering_median']> 10) & (orders_products_merge['ordering_median'] <= 20 ),'Customer_order_frequency'] = 'Regular customer'


In [30]:
orders_products_merge.loc[orders_products_merge['ordering_median'] <= 10 ,'Customer_order_frequency'] = 'Frequent customer'


In [31]:
orders_products_merge['Customer_order_frequency'].value_counts(dropna=False)

Customer_order_frequency
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
nan                             5
Name: count, dtype: int64

In [24]:
orders_products_merge[['user_id','ordering_median','Customer_order_frequency']].head(30)

,user_id,ordering_median,Customer_order_frequency
0,138,8.0,Frequent customer
1,138,8.0,Frequent customer
2,709,8.0,Frequent customer
3,764,9.0,Frequent customer
4,764,9.0,Frequent customer
5,777,11.0,Regular customer
6,825,20.0,Regular customer
7,910,6.0,Frequent customer
8,1052,10.0,Frequent customer
9,1052,10.0,Frequent customer


In [32]:
# Exporting to pickle file
orders_products_merge.to_pickle(os.path.join(data_path, '02 Data', 'Prepared Data', 'orders_products_merge_Grouped_Agged.pkl'))